# Question Answering with `RetrievalQA` chain

* Step 1: Retrieve relevant docs from vector DB
* Step 2: (Optional) Compress the relevant splits to fit into LLM context
* Step 3: Send the query + context to LLM to generate response

## Vectorstore retrieval
Start by loading Vector DB

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [4]:
print(vectordb._collection.count())

209


In [5]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [14]:
llm_name = "gpt-3.5-turbo"
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0) # set temperature to 0 when you want factual answers

### RetrievalQA chain

In [9]:
from langchain.chains import RetrievalQA

In [10]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [12]:
result = qa_chain.invoke({"query": question})

In [13]:
result["result"]

'The major topics for this class seem to include machine learning, statistics, and algebra. Additionally, there will be discussions on extensions of the material covered in the main lectures.'

### Prompt

In [15]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [16]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [17]:
question = "Is probability a class topic?"

In [18]:
result = qa_chain.invoke({"query": question})

In [19]:
result["result"]

'Yes, probability is a class topic assumed to be familiar to students, along with basic statistics and linear algebra. Thanks for asking!'

In [20]:
result["source_documents"][0]

Document(page_content="of this class will not be very program ming intensive, although we will do some \nprogramming, mostly in either MATLAB or Octa ve. I'll say a bit more about that later.  \nI also assume familiarity with basic proba bility and statistics. So most undergraduate \nstatistics class, like Stat 116 taught here at Stanford, will be more than enough. I'm gonna \nassume all of you know what ra ndom variables are, that all of you know what expectation \nis, what a variance or a random variable is. And in case of some of you, it's been a while \nsince you've seen some of this material. At some of the discussion sections, we'll actually \ngo over some of the prerequisites, sort of as  a refresher course under prerequisite class. \nI'll say a bit more about that later as well.  \nLastly, I also assume familiarity with basi c linear algebra. And again, most undergraduate \nlinear algebra courses are more than enough. So if you've taken courses like Math 51, \n103, Math 113 or 

### RetrievalQA chain types

In [25]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce" # instead of `stuff`
)

In [22]:
result = qa_chain_mr({"query": question})

In [23]:
result["result"]

'Yes, probability is a class topic mentioned in the text. The instructor assumes familiarity with basic probability and statistics, including concepts like random variables, expectation, variance, and random variables. The text also mentions that some discussion sections will go over prerequisites as a refresher course.'

If you wish to experiment on the `LangChain plus platform`:

 * Go to [langchain plus platform](https://www.langchain.plus/) and sign up
 * Create an API key from your account's settings
 * Use this API key in the code below   
 * uncomment the code  
 Note, the endpoint in the video differs from the one below. Use the one below.

### RetrievalQA limitations
 
QA fails to preserve conversational history.

In [26]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [28]:
question = "Is probability a class topic?"
result = qa_chain.invoke({"query": question})
result["result"]

'Yes, probability is a class topic in the course being described. The instructor assumes familiarity with basic probability and statistics, so it will likely be covered in the course.'

In [29]:
question = "why are those prerequesites needed?"
result = qa_chain.invoke({"query": question})
result["result"]

'The prerequisites of basic knowledge in computer science, probability and statistics, and linear algebra are needed for the machine learning class because they form the foundational understanding required to grasp the concepts and algorithms taught in the course. Understanding these subjects helps students apply machine learning algorithms effectively to real-world problems and conduct research in the field.'